# Tải và sử dụng mô hình đã fine-tune

- **Mục đích**:  
  - Kết nối Google Colab với Google Drive để truy cập mô hình đã lưu.  
  - Tải mô hình BioBERT đã fine-tune cho tác vụ SRL (Semantic Role Labeling).  
  - Thực hiện dự đoán trên câu mới và hiển thị kết quả theo dạng dễ đọc.  


- **Quy trình**

1. **Thiết lập môi trường**  
     - Kết nối Google Drive (`drive.mount`).  
     - Cài đặt thư viện `transformers` và `accelerate`.  

In [ ]:
# --- 1. Thiết lập môi trường ---
from google.colab import drive


print("Đang kết nối với Google Drive...")
# Kết nối Colab với Google Drive để truy cập mô hình đã lưu
drive.mount('/content/drive')
import os
print("\nĐang cài đặt thư viện transformers...")
# Cài đặt thư viện HuggingFace Transformers + Accelerate
!pip install transformers[torch] accelerate -q
print("Cài đặt hoàn tất!")

Đang kết nối với Google Drive...
Mounted at /content/drive

Đang cài đặt thư viện transformers...
 Cài đặt hoàn tất!



  2. **Tải mô hình từ Drive**  
     - Kiểm tra đường dẫn chứa mô hình đã lưu.  
     - Nếu tồn tại: tải pipeline với task `"ner"` (Token Classification).  
     - Dùng `aggregation_strategy="simple"` để tự động gộp sub-token thành từ hoàn chỉnh.  

In [ ]:
# --- 2. Tải mô hình từ Google Drive ---
from transformers import pipeline

# Đường dẫn tới mô hình Biobert đã fine-tune và lưu trước đó
final_model_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Finetuned_Models/biobert-srl-best-model'

# Kiểm tra xem mô hình có tồn tại ở đường dẫn hay không
if not os.path.isdir(final_model_path):
    print(f"LỖI: Không tìm thấy thư mục mô hình tại đường dẫn: {final_model_path}")
    print("Vui lòng kiểm tra lại đường dẫn và chắc chắn rằng bạn đã huấn luyện và lưu mô hình thành công.")
else:
    print(f"\nĐang tải pipeline từ: {final_model_path}")
    # Tải pipeline cho tác vụ NER (Token Classification)
    # aggregation_strategy="simple" giúp gom các sub-token thành một thực thể hoàn chỉnh
    srl_pipeline = pipeline(
        "ner",
        model=final_model_path,
        tokenizer=final_model_path,
        aggregation_strategy="simple"
    )
    print("Tải pipeline thành công! Mô hình đã sẵn sàng để dự đoán.")


Đang tải pipeline từ: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Finetuned_Models/biobert-srl-best-model


Device set to use cpu


 Tải pipeline thành công! Mô hình đã sẵn sàng để dự đoán.



  3. **Dự đoán trên câu mới**  
     - Nhập văn bản đầu vào vào biến `text_to_predict`.  
     - Chạy pipeline để lấy các entity (predicate và argument).  
     - Tách nhãn thành 2 phần: `Predicate` và `Argument`.  
     - In ra chi tiết: Predicate, Argument, Text, Confidence Score.  


In [ ]:
# --- 3. Sử dụng mô hình để dự đoán ---
# Thay đổi biến `text_to_predict` để test với câu khác
text_to_predict = "Besides these side chain interactions with the 06-alkyl group, structure-based analysis of mutational data suggests the ability of substitutions at Gly156 and Lys165 to confer resistance to 06-BG through backbone distortions."

print(f"\nCâu đầu vào: '{text_to_predict}'")

# Lấy kết quả dự đoán từ pipeline
results = srl_pipeline(text_to_predict)

# --- Xử lý và in kết quả cho dễ đọc ---
print("\nKết quả dự đoán các vai trò ngữ nghĩa (SRL):")
if not results:
    print("Mô hình không nhận dạng được argument nào.")
else:
    for entity in results:
        try:
            # Lấy nhãn entity_group, tách thành Predicate và Argument
            full_label = entity['entity_group']
            predicate, arg_label = full_label.split('-', 1)

            # In ra thông tin chi tiết của thực thể
            print(f"- Predicate: {predicate}")
            print(f"  - Argument: {arg_label}")
            print(f"  - Text: '{entity['word']}'")
            print(f"  - Confidence Score: {entity['score']:.4f}\n")
        except ValueError:
            # Nếu nhãn không đúng định dạng (không tách được) thì báo lỗi
            print(f"  - Không thể phân tích nhãn: {entity}")


Câu đầu vào: 'Besides these side chain interactions with the 06-alkyl group, structure-based analysis of mutational data suggests the ability of substitutions at Gly156 and Lys165 to confer resistance to 06-BG through backbone distortions.'

Kết quả dự đoán các vai trò ngữ nghĩa (SRL):
  - Predicate: confer.01
    - Argument: ARG-1
    - Text: 'the'
    - Confidence Score: 0.5007

  - Predicate: confer.01
    - Argument: ARG-0
    - Text: 'of substitutions at gly156 and lys165'
    - Confidence Score: 0.9854

  - Predicate: confer.01
    - Argument: ARG-1
    - Text: 'resistance'
    - Confidence Score: 0.9884

  - Predicate: confer.01
    - Argument: ARG-2
    - Text: '06 - bg'
    - Confidence Score: 0.9934



- **Kết quả**:  
  - Mô hình được tải thành công từ Google Drive.  
  - Câu đầu vào được phân tích và các vai trò ngữ nghĩa (SRL) được hiển thị rõ ràng.  
  - Nếu mô hình không nhận dạng được argument nào → in thông báo tương ứng.  